In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4
import seaborn as sb

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving voter_final.csv to voter_final.csv


In [ ]:
file_to_open = ("voter_final.csv")
ml_df = pd.read_csv(file_to_open)
ml_df.head()

,Voter_ID,Birth_Year,City,Zip_Code,Party,Registration_Date,2020_Primary_Election,2020_General_Election,2016_Primary_Election,2016_General_Election,Generational_Group
0,704844,1959,SEVEN HILLS,44131,NOPTY,1/1/1901,0,t,D,t,Boomers
1,1658758,1968,PARMA,44130,REP,9/9/1996,0,t,R,t,Generation X
2,2005536,1976,BROADVIEW HTS,44147,REP,3/8/2004,0,t,R,t,Generation X
3,2465762,1991,SOLON,44139,NOPTY,9/27/2010,0,f,0,f,Millennials
4,710633,1968,INDEPENDENCE,44131,REP,1/1/1901,0,t,R,t,Generation X


In [ ]:
party={"D": 0, "R": 1, "L": 2, "0": 3, "N": 3, " ": 3, "G": 4, "X": 3}
ml_df['2016_Primary_Election'] = ml_df['2016_Primary_Election'].apply(lambda x:party[x])
ml_df['2020_Primary_Election'] = ml_df['2020_Primary_Election'].apply(lambda x:party[x])


In [ ]:
# Handle Categorical "Party" columns
P_ID = {"DEM": 0, "REP": 1,"LIB": 2, "NOPTY": 3, "GRE": 4}
ml_df["Party"] = ml_df["Party"].apply(lambda x:P_ID[x])
ml_df.head()

,Voter_ID,Birth_Year,City,Zip_Code,Party,Registration_Date,2020_Primary_Election,2020_General_Election,2016_Primary_Election,2016_General_Election,Generational_Group
0,704844,1959,SEVEN HILLS,44131,3,1/1/1901,3,t,0,t,Boomers
1,1658758,1968,PARMA,44130,1,9/9/1996,3,t,1,t,Generation X
2,2005536,1976,BROADVIEW HTS,44147,1,3/8/2004,3,t,1,t,Generation X
3,2465762,1991,SOLON,44139,3,9/27/2010,3,f,3,f,Millennials
4,710633,1968,INDEPENDENCE,44131,1,1/1/1901,3,t,1,t,Generation X


In [ ]:
ml_df['Generational_Group'].unique()

array(['Boomers', 'Generation X', 'Millennials', 'Generation Z', 'Silent'],
      dtype=object)

In [ ]:
Group = {"Silent": 1,"Boomers": 2, "Generation X": 3, "Millennials": 4, "Generation Z": 5}
ml_df["Generational_Group"] = ml_df["Generational_Group"].apply(lambda x:Group[x])

In [ ]:
# Code the cities to unique #s
cities = {'SEVEN HILLS': 1, 'PARMA': 2, 'BROADVIEW HTS': 3, 'SOLON': 4, 'INDEPENDENCE': 5,
       'EUCLID': 6, 'CLEVELAND HTS': 7, 'GARFIELD HTS': 8, 'CLEVELAND': 9,
       'ROCKY RIVER': 10, 'WESTLAKE': 11, 'ORANGE': 12, 'PEPPER PIKE':13, 'MAPLE HTS':14,
       'EAST CLEVELAND':15, 'BROOK PARK':16, 'NORTH ROYALTON':17, 'HIGHLAND HTS':18,
       'WARRENSVILLE HTS':19, 'FAIRVIEW PARK':20, 'SOUTH EUCLID':21, 'BEREA':22,
       'MIDDLEBURG HTS':23, 'LAKEWOOD':24, 'LYNDHURST':25, 'SHAKER HTS':25,
       'BROOKLYN':27, 'OLMSTED FALLS':28, 'UNIVERSITY HTS':29, 'GATES MILLS':30,
       'MAYFIELD HTS':31, 'PARMA HTS':32, 'BEACHWOOD':33, 'RICHMOND HTS':33,
       'BRECKSVILLE':34, 'BEDFORD':35, 'BEDFORD HTS':36, 'OAKWOOD':37,
       'NORTH OLMSTED':38, 'OLMSTED TWP':39, 'STRONGSVILLE':40, 'BROOKLYN HTS':41,
       'NEWBURGH HTS':42, 'BAY VILLAGE':43, 'MORELAND HILLS':44, 'WALTON HILLS':45,
       'CUYAHOGA HTS':46, 'VALLEY VIEW':47, 'MAYFIELD VILLAGE':48, 'BENTLEYVILLE':49,
       'GLENWILLOW':50, 'CHAGRIN FALLS':51, 'HUNTING VALLEY':52, 'BRATENAHL':53,
       'WOODMERE':54, 'HIGHLAND HILLS':55, 'NORTH RANDALL':56, 'LINNDALE': 57,
       'CHAGRIN FALLS TWP': 58}

ml_df["City"] = ml_df["City"].apply(lambda x:cities[x])
# Set voter_id_org column as index

ml_df =ml_df.set_index('Voter_ID')
ml_df.head()


,Birth_Year,City,Zip_Code,Party,Registration_Date,2020_Primary_Election,2020_General_Election,2016_Primary_Election,2016_General_Election,Generational_Group
Voter_ID,,,,,,,,,,
704844,1959,1,44131,3,1/1/1901,3,t,0,t,2
1658758,1968,2,44130,1,9/9/1996,3,t,1,t,3
2005536,1976,3,44147,1,3/8/2004,3,t,1,t,3
2465762,1991,4,44139,3,9/27/2010,3,f,3,f,4
710633,1968,5,44131,1,1/1/1901,3,t,1,t,3


In [ ]:
#Replace True and False Values with 0 and 1
ml_df.replace(('f', 't'), (0, 1), inplace=True)

In [ ]:
ml_df['Registration_Date'] =  pd.to_datetime(ml_df['Registration_Date'], dayfirst=True)


In [ ]:
# Restrict to voters eligable to vote in 2016 and 2020 election (born 1998 or before)
ml_df = ml_df[ml_df['Birth_Year'] <= 1998]

In [ ]:
ml_df = ml_df[ml_df['Registration_Date'] <= '2016-10-05']

In [ ]:
ml_df_cleaned = ml_df.drop(['Registration_Date'], axis=1)


In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf


In [ ]:
ml_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 555189 entries, 704844 to 276903
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype
---  ------                 --------------   -----
 0   Birth_Year             555189 non-null  int64
 1   City                   555189 non-null  int64
 2   Zip_Code               555189 non-null  int64
 3   Party                  555189 non-null  int64
 4   2020_Primary_Election  555189 non-null  int64
 5   2020_General_Election  555189 non-null  int64
 6   2016_Primary_Election  555189 non-null  int64
 7   2016_General_Election  555189 non-null  int64
 8   Generational_Group     555189 non-null  int64
dtypes: int64(9)
memory usage: 42.4 MB


In [ ]:
# Remove 2020 General Election target from features data
y = ml_df_cleaned["2020_General_Election"]
X = ml_df_cleaned.drop(columns=["2020_General_Election"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=33)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
y_train_pred = rf_model.predict(X_train_scaled)
print(f" Random forest predictive accuracy(test): {accuracy_score(y_test,y_pred):.3f}")
print(f" Random forest predictive accuracy(train): {accuracy_score(y_train,y_train_pred):.3f}")


 Random forest predictive accuracy(test): 0.827
 Random forest predictive accuracy(train): 0.855


In [ ]:
param_grid = { 
    'n_estimators': [100, 300],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [2,6,10],
    'criterion' :['gini', 'entropy']
}

In [ ]:
CV_rfc = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train_scaled, y_train)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(n_estimators=128,
                                              random_state=33),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 6, 10],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [100, 300]})

In [ ]:
CV_rfc.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'max_features': 'log2',
 'n_estimators': 100}

In [ ]:
rfc1=RandomForestClassifier(random_state=33, max_features='log2', n_estimators= 100, max_depth=10, criterion='entropy')

In [ ]:
rfc1.fit(X_train_scaled, y_train)

RandomForestClassifier(criterion='entropy', max_depth=10, max_features='log2',
                       random_state=33)

In [ ]:
pred=rfc1.predict(X_test_scaled)
pred_train= rfc1.predict(X_train_scaled)

In [ ]:
print("Accuracy for Random Forest on CV data(test): ",accuracy_score(y_test,pred))
print("Accuracy for Random Forest on CV data(train): ",accuracy_score(y_train,pred_train))


Accuracy for Random Forest on CV data(test):  0.837007737863658
Accuracy for Random Forest on CV data(train):  0.839429766733671


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

report = classification_report(y_test, pred)
print(report)

              precision    recall  f1-score   support

           0       0.71      0.60      0.65     35164
           1       0.87      0.92      0.89    103634

    accuracy                           0.84    138798
   macro avg       0.79      0.76      0.77    138798
weighted avg       0.83      0.84      0.83    138798

